In [37]:
import sys
sys.path[1] = '/Users/vishwajit/.pyenv/versions/3.7.3/lib/python3.7/site-packages'
import tensorflow as tf
import numpy as np
import os
import pandas as pd
from tensorflow.keras import layers
from datetime import datetime
from matplotlib import pyplot as plt
from matplotlib import dates as md
import plotly.express as px
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Conv1D, Dense, Activation, Dropout, Lambda, Multiply, Add, Concatenate
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv1D,Conv2D,LSTM,GRU,Lambda
from tensorflow.keras.layers import MaxPooling1D, Bidirectional
from sklearn.metrics import accuracy_score
import datetime
from tqdm import tqdm
from sklearn.utils import shuffle
import os
from scipy import signal
import gc

In [2]:
converter = tf.lite.TFLiteConverter.from_saved_model('final_model/my_model')
lite_model = converter.convert()

TypeError: list indices must be integers or slices, not list

In [3]:
open("sensor_converted_model.tflite", "wb").write(lite_model)

348316

In [4]:
interpreter = tf.lite.Interpreter(model_path="sensor_converted_model.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

input_shape = input_details[0]['shape']
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()
interpreter.allocate_tensors()

output_data = interpreter.get_tensor(output_details[0]['index'])

In [31]:
input_shape

array([  1, 100,   5], dtype=int32)

In [5]:
df=pd.DataFrame()
df1 = pd.DataFrame()
os.chdir('/Users/vishwajit/Desktop/Driver Performance Analysis Files/labelled_driver_data')
for files in os.listdir(os.getcwd()):
    print(files)
    if files!='river4':
        os.chdir('/Users/vishwajit/Desktop/Driver Performance Analysis Files/labelled_driver_data/{}'.format(files))
        for filename in os.listdir(os.getcwd()):
            print(filename)
            if 'acelerometro' in filename:
                data = pd.read_csv(filename ,sep =',', header = 0 )
                data.columns=['timestamp','uptimeNanos','X','Y','Z']
                data['label'] = 0
                data['device'] = 'acc'
                data['uptimeNanos'] /= 1e9
            elif 'giro' in filename:
                data2 = pd.read_csv(filename ,sep =',', header = 0 )
                data2.columns=['timestamp','uptimeNanos','X','Y','Z']
                data2['label'] = 0
                data2['device'] = 'gyr'
                data2['uptimeNanos'] /= 1e9
            elif 'ground' in filename:
                labels = pd.read_csv(filename , sep=',')
                labels[' inicio'] += data['uptimeNanos'][0]
                labels[' fim'] += data['uptimeNanos'][0]

        for event,start, end in labels.values:
            #print(data2.loc[data2.uptimeNanos.between(start, end),['label']].shape[0])
            data.loc[data.uptimeNanos.between(start, end),['label']] = [event]
            data2.loc[data2.uptimeNanos.between(start, end),['label']] = [event]
        df1 = pd.concat([data,data2], axis =1 , ignore_index=True)
        df = df.append(df1)
        del data, data2, labels


driver1
aceleracaoLinear_terra.txt
giroscopio_terra.txt
acelerometro_terra.txt
groundTruth.txt
driver2
aceleracaoLinear_terra.txt
giroscopio_terra.txt
acelerometro_terra.txt
groundTruth.txt
driver4
aceleracaoLinear_terra.txt
giroscopio_terra.txt
acelerometro_terra.txt
groundTruth.txt
driver3
aceleracaoLinear_terra.txt
giroscopio_terra.txt
acelerometro_terra.txt
groundTruth.txt


In [6]:
df.columns =['timedate','time','X_acc','Y_acc','Z_acc','label','device','s','s1','X_gyr','Y_gyr','Z_gyr','s2','s3']
df.timedate = pd.to_datetime(df.timedate)
df['label'] = df.label.apply(lambda x : 0 if x in ['evento_nao_agressivo',0] else 1)

In [7]:
df['time'] = df.timedate.apply(lambda x : x.time())
print('The sampling frequency in Hz is:',df.groupby('time').X_gyr.count().mean())

The sampling frequency in Hz is: 50.8816644993498


In [8]:
df = df[['timedate','time','X_acc','Y_acc','Z_acc','X_gyr','Y_gyr','Z_gyr','label']]

In [9]:
df.head()

,timedate,time,X_acc,Y_acc,Z_acc,X_gyr,Y_gyr,Z_gyr,label
0,2016-05-14 11:17:09,11:17:09,-0.047600,0.004670,9.852109,0.030578,-0.035392,-0.033423,0
1,2016-05-14 11:17:09,11:17:09,-0.066993,0.060984,9.634208,-0.007943,-0.017933,-0.022102,0
2,2016-05-14 11:17:09,11:17:09,-0.200131,0.170454,9.512366,-0.005299,-0.041667,-0.008381,0
3,2016-05-14 11:17:09,11:17:09,-0.185315,-0.039818,9.671576,0.020004,0.012221,0.001138,0
4,2016-05-14 11:17:09,11:17:09,-0.011559,0.049461,9.844943,-0.025175,0.020601,0.023692,0


In [10]:
df.dropna(inplace = True)
dftrain = df.copy()
def train_dataset(series,window_size, batch_size):
    ds = tf.data.Dataset.from_tensor_slices((series))
    ds = ds.window(window_size, shift=20, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size))
    ds = ds.shuffle(10000).map(lambda window: (window[:,:-1], window[50,-1:]))
    #ds = ds.batch(batch_size).prefetch(1)
    return ds
train= train_dataset(dftrain[['X_acc','Y_acc','X_gyr','Y_gyr','Z_gyr','label']].values,100,128)

In [11]:
X=[]
y=[]
for i,j in train:
    X.append(i.numpy())
    y.append(j.numpy())
X = np.asarray(X)
y = np.asarray(y)

In [36]:
y_pred = np.zeros(X.shape[0])
i = 0
for x in X:   
    interpreter.allocate_tensors()
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    x = x.reshape(1,100,5)
    input_data = np.array(x, dtype=np.float32)
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    y_pred[i] = output_data[0][0]
    i+=1

In [42]:
y_pred = [0 if p<0.5 else 1 for p in y_pred]
accuracy_score(y,y_pred)

0.9707198567958062